In [72]:
import os.path

archive_name = 'hackathon_GS_Labs.zip'

! gdown --id 13S14yNg4_-CCBr4QK4jvfCEDOn6yVYEK
! unzip -q -o "$archive_name"
! rm "$archive_name"

Downloading...
From: https://drive.google.com/uc?id=13S14yNg4_-CCBr4QK4jvfCEDOn6yVYEK
To: /content/hackathon_GS_Labs.zip
6.63MB [00:00, 57.5MB/s]


In [73]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.7/dist-packages (1.2.4)


In [74]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [104]:
with open('USER_UID.txt') as f:
    ids = [int(id) for id in f.read().split(', ')]

In [76]:
content = pd.read_csv('content.csv')
content = content.set_index('content_uid')
content

AttributeError: ignored

                            name  ... duration_seconds
content_uid                       ...                 
9882         "Шоу "Студия "Союз"  ...              NaN
13701        "Шоу "Студия "Союз"  ...           2823.0
12274        "Шоу "Студия "Союз"  ...           2694.0
13579        "Шоу "Студия "Союз"  ...           2739.0
18055        "Шоу "Студия "Союз"  ...           2887.0
...                          ...  ...              ...
10190               Ящик Пандоры  ...           2674.0
669                 Ящик Пандоры  ...           2674.0
900                 Ящик Пандоры  ...           2633.0
679                 Ящик Пандоры  ...           2678.0
934                 Ящик Пандоры  ...           2744.0

[17297 rows x 5 columns]

In [77]:
# находим общую длительность для каждого сериала
# serial_sums = content.groupby('serial_id')['duration_seconds'].sum()

# serial_sums.rename_axis('content_uid', inplace=True)
# a = content.join(serial_sums, on=['content_uid'], rsuffix='_r')
# a

"""
Нужно:
  1. Для каждого сериала в duration_seconds положить значения из duration_seconds_r
  2.
"""

'\nНужно:\n  1. Для каждого сериала в duration_seconds положить значения из duration_seconds_r\n  2.\n'

In [78]:
watch_history = pd.read_csv('watch_history.csv')

def get_watch_history_by_id(id):
  return watch_history[watch_history['user_uid'] == id]

watch_history

AttributeError: ignored

         user_uid  content_uid  second
0               1        26719     400
1               1        11882     627
2               1        26174    5693
3               4        28436     828
4               4         3836      64
...           ...          ...     ...
1286126   1302881        26454    5781
1286127   1302912        25224      16
1286128   1302913        27762     118
1286129   1302914        28136      70
1286130   1302914        24797      26

[1286131 rows x 3 columns]

In [79]:
joined = watch_history.join(content, how='inner', on=['content_uid'])
joined_reduced = joined.iloc[::15]

### нужные колонки для фильтарции от шума.
["user_uid", "content_uid", "second", "duration_seconds"]


In [80]:
def build_dataset(df):
  dataset = df.copy()
  dataset['watch_coeff'] = dataset['second'] / dataset['duration_seconds']
  return dataset
  
dataset = build_dataset(joined_reduced)
dataset

AttributeError: ignored

         user_uid  content_uid  ...  duration_seconds watch_coeff
0               1        26719  ...            6491.0    0.061624
23952       24204        26719  ...            6491.0    0.974734
53998       54530        26719  ...            6491.0    0.984902
82798       83738        26719  ...            6491.0    0.968726
110882     112152        26719  ...            6491.0    0.983208
...           ...          ...  ...               ...         ...
1192656   1208452        23512  ...            1372.0    0.897230
1203784   1219858         4690  ...            2475.0    0.985455
1214772   1231015        24970  ...            5019.0    0.000797
1228628   1245038        13087  ...            1513.0    0.999339
1285065   1301845        24993  ...             825.0    1.000000

[85743 rows x 9 columns]

In [81]:
#dataset[dataset['user_uid'] == 844861]

In [82]:
# Просмотр кол-ва типов, которые пересматриваются или нет
dataset[dataset['watch_coeff'] < 1.0]['type'].value_counts()

movie      56127
episode    20813
Name: type, dtype: int64

In [83]:
def pivot(a):
  a = None
  a = dataset.groupby(['user_uid','content_uid'])['watch_coeff'].max()
  a = a.unstack()
  return a

In [84]:
fd = None
fd = dataset.pivot(index='user_uid', columns='content_uid', values='watch_coeff')
fd.fillna(0, inplace=True)

print(len(fd))

films_watched = dataset.groupby('content_uid')['second'].agg('count')
users_watched = dataset.groupby('user_uid')['second'].agg('count')

fd = fd.loc[:, films_watched[films_watched > 25].index]
fd = fd.loc[users_watched[users_watched > 10].index, :]
fd

49503


AttributeError: ignored

content_uid  321    345    462    472    ...  30283  30284  30331  30649
user_uid                                 ...                            
132            0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
638            0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1848           0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
5548           0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
7431           0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
...            ...    ...    ...    ...  ...    ...    ...    ...    ...
1240398        0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1247831        0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1250019        0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1252114        0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1259513        0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0

[425 rows x 758 columns]

In [85]:
csr_data = csr_matrix(fd.values)
fd.reset_index(inplace=True)
print(csr_data)

  (1, 36)	1.0
  (1, 63)	1.0
  (1, 306)	1.0
  (1, 310)	0.026506395438434274
  (1, 524)	1.0
  (2, 14)	0.016439721022915976
  (2, 73)	0.9987323050919079
  (2, 231)	0.9675310525168882
  (2, 293)	0.43521437578814626
  (2, 337)	0.8989970086222066
  (2, 428)	0.02058016464131713
  (2, 462)	0.049342770475227504
  (2, 475)	0.9210440456769984
  (2, 559)	0.010190319196763074
  (2, 619)	0.11042084168336673
  (2, 652)	0.48273797645180605
  (2, 726)	0.3258488499452355
  (2, 729)	0.09415407029684938
  (2, 746)	0.07244391162870355
  (3, 79)	1.0
  (4, 130)	0.9416909620991254
  (4, 394)	0.12068640392230813
  (4, 450)	0.043785588062045945
  (5, 75)	0.9442431561996779
  (5, 519)	0.9602194787379973
  :	:
  (419, 714)	0.9940298507462687
  (419, 756)	0.16779026217228465
  (420, 20)	0.9516529715268488
  (420, 64)	0.0012156576707999028
  (420, 395)	0.8336137595812302
  (421, 33)	0.9882697947214076
  (421, 45)	0.9993318485523385
  (422, 39)	0.9749753971601294
  (422, 155)	0.9453698728282286
  (422, 249)	0.552785

In [86]:
print(csr_data[420])

  (0, 20)	0.9516529715268488
  (0, 64)	0.0012156576707999028
  (0, 395)	0.8336137595812302


In [87]:
# fd.reset_index()
# fd
# content[content.index == 1282]

AttributeError: ignored

content_uid  user_uid  321  345  462  472  ...  30163  30283  30284  30331  30649
0                 132  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
1                 638  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
2                1848  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
3                5548  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
4                7431  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
..                ...  ...  ...  ...  ...  ...    ...    ...    ...    ...    ...
420           1240398  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
421           1247831  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
422           1250019  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
423           1252114  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0
424           1259513  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0

[425 rows x 759

In [88]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [89]:
# дописать рекомендатор из статьи https://www.analyticsvidhya.com/blog/2020/11/create-your-own-movie-movie-recommendation-system/

In [90]:
fd[fd['user_uid'] == 1240398]

AttributeError: ignored

content_uid  user_uid  321  345  462  472  ...  30163  30283  30284  30331  30649
420           1240398  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0    0.0    0.0

[1 rows x 759 columns]

In [91]:
def get_movie_recommendation(user_id, n_movies_to_reccomend=10):
  user_idx = fd[fd['user_uid'] == user_id].index[0]
  distances, indices = knn.kneighbors(csr_data[user_idx],n_neighbors=n_movies_to_reccomend+1)
  rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
  res = []
  for val in rec_movie_indices:
      user_idx = fd.iloc[val[0]]['user_uid']
      res.append(user_idx)
  return res

In [118]:
# ids = [1240398]

def get_user_recommendation(user_uid, rec_count=5):
  # dataset
  return dataset[dataset.user_uid==user_uid].sort_values(by=['watch_coeff'], ascending=False)\
  .head(n=rec_count)['content_uid']
  

with open('recommendations.csv', 'w') as f:
    lines = []
    for id in ids:
        users = get_movie_recommendation(id)
        line = str(id) + ','
        for u in users:
          usr = get_movie_recommendation(user_id=id)
          line += ','.join(str(i) for i in usr) + ','
        line += '\n'
        lines.append(line)
    f.writelines(lines)



AttributeError: ignored

         user_uid  content_uid  ...  duration_seconds watch_coeff
0               1        26719  ...            6491.0    0.061624
23952       24204        26719  ...            6491.0    0.974734
53998       54530        26719  ...            6491.0    0.984902
82798       83738        26719  ...            6491.0    0.968726
110882     112152        26719  ...            6491.0    0.983208
...           ...          ...  ...               ...         ...
1192656   1208452        23512  ...            1372.0    0.897230
1203784   1219858         4690  ...            2475.0    0.985455
1214772   1231015        24970  ...            5019.0    0.000797
1228628   1245038        13087  ...            1513.0    0.999339
1285065   1301845        24993  ...             825.0    1.000000

[85743 rows x 9 columns]